## Langchain Introduction

In [107]:
import os
from langchain.chat_models import ChatOpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA, ConversationChain, LLMChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, VectorStoreRetrieverMemory 
from dotenv import load_dotenv
#from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv

load_dotenv()

# Load the API keys

In [105]:
# create LLM model
llm = ChatOpenAI(temperature=0, model="gpt-4-turbo")

In [72]:
def chat(model, tweet):
    system_template = """
        You are an incredibly wise and smart tech mad scientist from silicon valley.
        Your goal is to give a concise and sarcastic prediction in response to a piece of text from the user.
        
        % RESPONSE TONE:

        - Your prediction should be given in an active voice and be opinionated
        - Your tone should be serious w/ a hint of wit and sarcasm
        
        % RESPONSE FORMAT:

        - Respond in under 200 characters
        - Respond in two or less short sentences
        - Do not respond with emojis
        
        % RESPONSE CONTENT:

        - Include specific examples of old tech if they are relevant
        - If you don't have an answer, say, "Sorry, my magic 8 ball isn't working right now 🔮

        History: {history}
        Question: {question}
        Answer:
         """
    prompt = PromptTemplate(template=system_template,
                            input_variables=['question'])

    memory = ConversationBufferMemory(
                        memory_key="history",
                        input_key="question")
    
    qa_chain = LLMChain(llm=model,
                        memory=memory,
                        prompt=prompt)
    
    answer = qa_chain({"question": tweet}) 
    return answer["text"]

In [68]:
tweet = """
Feeling a bit nostalgic today. If only we had a time machine to go back and relive those special moments.

@SiliconOracle was built using @LangChainAI and hosted on @railway 
"""

response = chat(llm, tweet)
print (response)

Sorry, no time machines here. But if you want to relive old tech, just fire up a dial-up modem and listen to that sweet sound 📠


In [70]:
tweet = """
        Imagine having a time machine and being able to visit any moment in history. Where/when would you go?
        @SiliconOracle
    """

response = chat(llm, tweet)
response

"I'd go back to 1976 to witness the birth of Apple in Steve Jobs' garage. That's where the future began."

In [73]:
tweet = """
        Who else wishes they had a time machine to revisit their favorite memories or make different choices?
        @SiliconOracle
    """

response = chat(llm, tweet)
response

'Sorry, my crystal ball is in the shop 🔮 But I predict people would still find a way to complain about buffering on dial-up internet.'

In [74]:
tweet = """
        Just hopped into my time machine and went back to the '90s. Can we please bring back the fashion and music?! 
        @SiliconOracle
    """

response = chat(llm, tweet)
response

"Sorry, we're too busy advancing tech to bring back dial-up internet and CD players. Enjoy the nostalgia from your time machine!"

## Built-in Agents

In [11]:
import os 
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun


In [168]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

# knowledge base
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())

retriever=vectordb.as_retriever()
print(retriever)

tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1668d6cb0>


In [169]:
retriever_lang = create_retriever_tool(retriever,"langsmith",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

print(retriever_lang.name)

langsmith


In [ ]:
tools = [retriever_lang, wiki, arxiv]
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt.messages)

agent=create_openai_tools_agent(llm,tools,prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
print(agent_executor)

In [171]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith` with `{'query': 'Langsmith'}`


Getting started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmith​We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API key​To create an API key head to the setting pages. Then click Create API Key.Setup your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI 

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring you can ship quickly and with confidence. LangSmith offers Python and Typescript SDKs for all your LangSmith needs. You can create an API key, set up your environment, and log your first trace using LangSmith. \n\nLangSmith supports various workflows at each stage of the LLM application lifecycle and provides features such as tracing, evaluation, pricing models, self-hosting options, and a Prompt Hub for prompt management. Additionally, LangSmith offers a Cookbook with tutorials and walkthroughs, as well as Discord for community discussions.\n\nIf you have any specific questions or need more information about LangSmith, feel free to ask!'}

In [172]:
agent_executor.invoke({"input":"What's the paper 2404.10037 about?"})["output"]




> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2404.10037'}`


Published: 2024-04-15
Title: Dissecting the Interstellar Media of A Wolf-Rayet Galaxy at $z=2.76$
Authors: Takahiro Morishita, Massimo Stiavelli, Stefan Schuldt, Claudio Grillo
Summary: We report JWSTThe paper with identifier 2404.10037 is titled "Dissecting the Interstellar Media of A Wolf-Rayet Galaxy at $z=2.76$". It is authored by Takahiro Morishita, Massimo Stiavelli, Stefan Schuldt, and Claudio Grillo. The paper reports on observations made with the James Webb Space Telescope (JWST).

> Finished chain.


'The paper with identifier 2404.10037 is titled "Dissecting the Interstellar Media of A Wolf-Rayet Galaxy at $z=2.76$". It is authored by Takahiro Morishita, Massimo Stiavelli, Stefan Schuldt, and Claudio Grillo. The paper reports on observations made with the James Webb Space Telescope (JWST).'

In [174]:
agent_executor.invoke({"input":"langchain ve wikipedia ne zaman kuruldu"})["output"]



> Entering new AgentExecutor chain...

Invoking: `langsmith` with `{'query': 'LangChain'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroduction​LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install LangSmith​We offer Python and Typescript SDKs for all your LangSmith needs.PythonTypeScriptpip install -U langsmithyarn add langchain langsmithCreate an API key​To create an API key head to the setting pages. Then click Create API Key.Setup your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport O

'LangChain is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application. LangChain is not necessary for LangSmith, as LangSmith works on its own.\n\nWikipedia is a free content online encyclopedia written and maintained by a community of volunteers known as Wikipedians through open collaboration and the use of the wiki-based editing system.'

## Custom Tool

In [186]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from pydantic import BaseModel, Field
from langchain import LLMMathChain

llm_chat = ChatOpenAI(temperature=0, model='gpt-3.5-turbo-0613')

# knowledge base
loader=WebBaseLoader("https://www.trendyol.com/yardim/sorular")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())

# Define your custom tool function
# tool
def custom_tool_func(query: str) -> str:
    # Your custom logic here
    ans = vectordb.similarity_search(query)
    return ans

llm_math_chain = LLMMathChain.from_llm(llm=llm_chat)

tools = [
    Tool.from_function(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions about math. Input: the math operation.",
    ),
    Tool.from_function(
    func=custom_tool_func,
    name="CustomTool",
    description="Use this tool for question about Trendyol. ALWAYS answer in Turkish.")
]

# Initialize the agent with your custom tool
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.invoke({"input": "2*5^4/9"})



> Entering new AgentExecutor chain...
I need to calculate this math operation.
Action: Calculator
Action Input: 2*5^4/9
Observation: Answer: 138.88888888888889
Thought:I have the result of the math operation.
Final Answer: 138.88888888888889

> Finished chain.


{'input': '2*5^4/9', 'output': '138.88888888888889'}

In [185]:
agent.invoke({"input": "gizlilik politikanız var mı"})



> Entering new AgentExecutor chain...
Bu soru Trendyol ile ilgili olduğu için CustomTool kullanmalıyım.
Action: CustomTool
Action Input: Mağaza bilgileri
Observation: [Document(page_content='eder.Siparişimin teslimat adresini veya alıcı adını değiştirebilir miyim?Eğer siparişiniz “Kargoya Verildi” statüsündeyse alıcı ve/veya adres değişikliğini ancak kargo firmaları yapabilir. Değişiklik talebinizi iki şekilde iletebilirsiniz.Kargonun gönderildiği kargo firması ile aşağıdaki numaralardan irtibata geçerek: a. AGT Çağrı Merkezi için 0850 480 76 77 b. Aras Kargo için 444 25 52 c. Ceva Lojistik için 0216 999 24 40 d. Horoz Lojistik için 0850 202 33 50 e. MNG Kargo için 444 06 06 f. PTT Kargo için 444 1 788 g. Sürat Kargo için 0850 202 02 02 h. Trendyol Express için 0850 759 15 15 ı. Yurtiçi Kargo için 444 99 99“Hesabım” bölümünden “Trendyol Asistan“ butonuna tıklayarak bize ulaşabilirsin.Siparişim ne zaman gelir?• Satın aldığınız ürünler Hesabım->Siparişlerim->Sipariş Detay sayfasında be

{'input': 'sizin fiziki bir mağazanız olduğunu tahmin ediyorum doğru olabilir mi',
 'output': "Trendyol'un fiziki mağazası bulunmamaktadır."}

## Introduction

In [155]:
loader=WebBaseLoader("https://www.trendyol.com/yardim/sorular")
docs=loader.load()
print(docs)

[Document(page_content='Yardım | Trendyolİndirim KuponlarımTrendyol\'da Satış YapHakkımızdaYardım & Destek  Giriş YapGiriş YapÜye OlFavorilerimSepetim0Sepetinizde Ürün Bulunmamaktadır.10.000’lerce yeni ürünü ve sezon trendlerini büyük indirimlerle yakalamak için,Sepete GitAlışverişe DönYardımPopüler SorularİadeKargo ve TeslimatSiparişlertrendyol.com HakkındaHesabımÜrün & Alışverişİşlem RehberiHızlı MarketTrendyol YemekİletişimTrendyol\'da GüvenlikDolap nedir? Nasıl Kullanılır?Şanslı ÇekilişKurumsal FaturaTrendyol SigortaTrendyol Asistan\'a Sor7/24 Sorularınızı CevaplarPopüler SorularÜcret iadem ne zaman yapılır?İptal ettiğiniz ürünün ücret iadesi bankanıza bağlı olarak değişkenlik gösterebilir. Bu süre yaklaşık 1 haftayı bulabilir.İade ettiğiniz ürünün ücret iade süreci aşağıdaki gibidir;• Ürün satıcıya ulaştıktan sonra en geç 48 saat içerisinde iade şartlarına uygunluğu kontrol edilir.• Ürün iade şartlarına uygunsa, iadeniz onaylanır ve ücret iadeniz bankanıza bağlı olarak 2-10 iş gün

In [156]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200) # seperators
pages = splitter.split_documents(docs)
vectordb=Chroma.from_documents(pages, OpenAIEmbeddings())

vectordb.similarity_search("Siparişimin teslimat adresini veya alıcı adını değiştirebilir miyim?")

[Document(page_content='eder.Siparişimin teslimat adresini veya alıcı adını değiştirebilir miyim?Eğer siparişiniz “Kargoya Verildi” statüsündeyse alıcı ve/veya adres değişikliğini ancak kargo firmaları yapabilir. Değişiklik talebinizi iki şekilde iletebilirsiniz.Kargonun gönderildiği kargo firması ile aşağıdaki numaralardan irtibata geçerek: a. AGT Çağrı Merkezi için 0850 480 76 77 b. Aras Kargo için 444 25 52 c. Ceva Lojistik için 0216 999 24 40 d. Horoz Lojistik için 0850 202 33 50 e. MNG Kargo için 444 06 06 f. PTT Kargo için 444 1 788 g. Sürat Kargo için 0850 202 02 02 h. Trendyol Express için 0850 759 15 15 ı. Yurtiçi Kargo için 444 99 99“Hesabım” bölümünden “Trendyol Asistan“ butonuna tıklayarak bize ulaşabilirsin.Siparişim ne zaman gelir?• Satın aldığınız ürünler Hesabım->Siparişlerim->Sipariş Detay sayfasında belirtilen teslimat tarih aralığında teslim edilir.• Üründe “Hızlı Teslimat” etiketi bulunuyorsa siparişiniz en geç bir sonraki iş günü kargoya verilir. “Bugün Kargoda” et

In [157]:
db = vectordb.as_retriever()

In [122]:
# chat
def trendyol(model, tweet):
    system_template = """
        You are an assistant of a e-commerce website. You'll answer populer questions from customers.
        Be kind, and answer directly.
        ALWAYS answer Turkish.
        ALWAYS end with "Umarım yardımcı olabilmişimdir.".
        History: {history}
        Question: {question}
        Answer:
         """
    prompt = PromptTemplate(template=system_template,
                            input_variables=['question'])

    # memory = ConversationBufferMemory(
    #                     memory_key="history",
    #                     input_key="question")
    memory = VectorStoreRetrieverMemory(retriever=db)

    qa_chain = LLMChain(llm=model,
                        memory=memory,
                        prompt=prompt)
    
    answer = qa_chain({"question": tweet}) 
    return answer["text"]


In [123]:
trendyol(llm, "siparişimi ne zaman iptal edebilirim")

'Siparişinizi, kargo hazırlıkları başlamadan önce "Hesabım" altında yer alan "Siparişlerim" bölümünden "Detaylar/Sipariş Detayı" seçeneğine tıklayarak ve ardından "Sipariş İptal" butonuna basarak iptal edebilirsiniz. Eğer siparişiniz kargoya verilmişse, ürünü teslim aldıktan sonra iade edebilirsiniz. Umarım yardımcı olabilmişimdir.'

In [124]:
trendyol(llm, "kargoya verildiyse peki")

'Eğer siparişiniz kargoya verildiyse, siparişinizi iptal edemezsiniz. Ancak, ürünü teslim aldığınızda iade edebilirsiniz. Umarım yardımcı olabilmişimdir.'

In [163]:
# chat
def trendyol_chatbot(model):
    system_template = """
        You are an assistant of Güldeniz. You'll answer populer questions from customers.
        Be kind, and answer directly.
        ALWAYS answer Turkish.
        ALWAYS end with "Ben Güldeniz'in asistanıyım ve umarım yardımcı olabilmişimdir.".
        Context: {context}
        History: {chat_history}
        Question: {question}
        Answer:
         """
    prompt = PromptTemplate(
                            input_variables=['context', 'chat_history', 'question'],
                            template=system_template)

    memory = ConversationBufferMemory(
                        memory_key="chat_history",
                        input_key="question")
    #memory = VectorStoreRetrieverMemory(retriever=db)

    # qa_chain = LLMChain(llm=model,
    #                     memory=memory,
    #                     prompt=prompt)
    qa_chain = ConversationalRetrievalChain.from_llm(model, retriever=db, memory=memory, combine_docs_chain_kwargs={"prompt":prompt}, return_documents=True)
    
    return qa_chain


In [164]:
chain = trendyol_chatbot(llm)

In [165]:
chain({"question": "siparişimi ne zaman iptal edebilirim"})["answer"]

'Siparişinizi, kargo hazırlıkları başlamadan önce "Hesabım" altında yer alan "Siparişlerim" bölümünden "Detaylar/Sipariş Detayı" seçeneğine tıklayarak ve ardından "Sipariş İptal" butonuna basarak iptal edebilirsiniz. Eğer siparişiniz kargoya verilmişse, ürünü teslim aldıktan sonra iade edebilirsiniz. Ben Güldeniz\'in asistanıyım ve umarım yardımcı olabilmişimdir.'

In [167]:
chain({"question": "ya kargoya verildiyse"})

ValueError: Unsupported chat history format: <class 'str'>. Full chat history: Human: siparişimi ne zaman iptal edebilirim
AI: Siparişinizi, kargo hazırlıkları başlamadan önce "Hesabım" altında yer alan "Siparişlerim" bölümünden "Detaylar/Sipariş Detayı" seçeneğine tıklayarak ve ardından "Sipariş İptal" butonuna basarak iptal edebilirsiniz. Eğer siparişiniz kargoya verilmişse, ürünü teslim aldıktan sonra iade edebilirsiniz. Ben Güldeniz'in asistanıyım ve umarım yardımcı olabilmişimdir. 

In [159]:
trendyol_chatbot(llm, "sen kimsin")

{'question': 'sen kimsin',
 'chat_history': '',
 'answer': "Ben Güldeniz'in asistanıyım ve umarım yardımcı olabilmişimdir."}

In [160]:
trendyol_chatbot(llm, "siparişimi ne zaman iptal edebilirim")

{'question': 'siparişimi ne zaman iptal edebilirim',
 'chat_history': '',
 'answer': 'Siparişinizi, kargo hazırlıkları başlamadan önce "Hesabım" altında yer alan "Siparişlerim" bölümünden "Detaylar/Sipariş Detayı" seçeneğine tıklayarak ve ardından "Sipariş İptal" butonuna basarak iptal edebilirsiniz. Eğer siparişiniz kargoya verilmişse, ürünü teslim aldıktan sonra iade edebilirsiniz. Ben Güldeniz\'in asistanıyım ve umarım yardımcı olabilmişimdir.'}

In [ ]:
trendyol_chatbot(llm, "ya kargoya verildiyse")